In [19]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module


In [20]:
stocks = pd.read_csv('C:/Users/pconn/OneDrive/Desktop/STONKS/Projects/constituents.csv')
stocks.head()

,Symbol,Name,Sector
0,MMM,3M Company,Industrials
1,AOS,A.O. Smith Corp,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie Inc.,Health Care
4,ABMD,ABIOMED Inc,Health Care


In [3]:
symbol='AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token=pk_98a9722b99584ee7b7c5ebed778846fc'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2115300096000,
 'week52high': 144.87,
 'week52low': 52.74,
 'week52highSplitAdjustOnly': 145.09,
 'week52lowSplitAdjustOnly': 53.15,
 'week52change': 0.7033564232759602,
 'sharesOutstanding': 16788096000,
 'float': 0,
 'avg10Volume': 93420713,
 'avg30Volume': 98447035,
 'day200MovingAvg': 122.4,
 'day50MovingAvg': 133.32,
 'employees': 147000,
 'ttmEPS': 3.68,
 'ttmDividendRate': 0.8180094787439657,
 'dividendYield': 0.006492138720190204,
 'nextDividendDate': '',
 'exDividendDate': '2021-02-05',
 'nextEarningsDate': '2021-01-27',
 'peRatio': 33.08775373064289,
 'beta': 1.1425318041271417,
 'maxChangePercent': 47.27771178972374,
 'year5ChangePercent': 4.593238396249867,
 'year2ChangePercent': 1.9748202712752772,
 'year1ChangePercent': 0.6224464174194089,
 'ytdChangePercent': -0.048999184856444056,
 'month6ChangePercent': 0.016370830157174554,
 'month3ChangePercent': 0.07540739590099088,
 'month1ChangePercent': -0.09262759924385644,
 'day30Chang

In [4]:
data['year1ChangePercent']

0.6224464174194089

In [21]:

# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Symbol'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [22]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token=pk_98a9722b99584ee7b7c5ebed778846fc'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,MMM,175.420,0.163132,N/A
1,AOS,61.680,0.431193,N/A
2,ABT,121.680,0.423157,N/A
3,ABBV,106.400,0.187705,N/A
4,ABMD,319.645,0.935648,N/A
...,...,...,...,...
492,YUM,104.810,0.034333,N/A
493,ZBRA,496.300,1.048366,N/A
494,ZBH,162.940,0.026864,N/A
495,ZION,54.210,0.218065,N/A


In [23]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,FCX,37.625,2.189280,N/A
1,CARR,37.090,2.108333,N/A
2,VIAC,64.610,1.436419,N/A
3,PYPL,265.360,1.291824,N/A
4,ALGN,570.400,1.226773,N/A
5,LB,51.590,1.108434,N/A
6,SIVB,522.200,1.053133,N/A
7,ZBRA,496.300,1.048366,N/A
8,KLAC,319.250,0.993336,N/A
9,URI,300.210,0.971402,N/A


In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:30000
30000


In [9]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

C:\Users\pconn\anaconda3\envs\python3newenv\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,FCX,37.480,2.189280,15
1,CARR,37.015,2.108333,15
2,VIAC,64.450,1.436419,9
3,PYPL,265.560,1.291824,2
4,ALGN,570.800,1.226773,1
5,LB,51.320,1.108434,11
6,SIVB,521.440,1.053133,1
7,ZBRA,495.350,1.048366,1
8,KLAC,315.960,0.993336,1
9,URI,299.990,0.971402,1


In [24]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token=pk_98a9722b99584ee7b7c5ebed778846fc'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

In [25]:
hqm_dataframe.fillna(value=0, inplace=True)

In [26]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0      0.567404
1      0.851107
2       0.84507
3      0.615694
4      0.975855
         ...   
492    0.394366
493    0.985915
494    0.380282
495    0.657948
496    0.521127
Name: One-Year Return Percentile, Length: 497, dtype: object
0      0.370221
1      0.607646
2      0.511066
3      0.442656
4      0.247485
         ...   
492    0.372233
493    0.927565
494    0.474849
495    0.901408
496    0.185111
Name: Six-Month Return Percentile, Length: 497, dtype: object
0      0.325956
1      0.509054
2      0.513078
3      0.450704
4      0.714286
         ...   
492    0.287726
493    0.901408
494    0.492958
495    0.915493
496    0.158954
Name: Three-Month Return Percentile, Length: 497, dtype: object
0      0.609658
1      0.698189
2      0.736419
3      0.215292
4      0.078471
         ...   
492    0.309859
493    0.947686
494    0.352113
495    0.820926
496    0.285714
Name: One-Month Return Percentile, Length: 497, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,175.420,N/A,0.163132,0.567404,0.107584,0.370221,0.027077,0.325956,0.050469,0.609658,N/A
1,AOS,61.720,N/A,0.431193,0.851107,0.276189,0.607646,0.108701,0.509054,0.076070,0.698189,N/A
2,ABT,121.730,N/A,0.423157,0.84507,0.205604,0.511066,0.109601,0.513078,0.086051,0.736419,N/A
3,ABBV,106.390,N/A,0.187705,0.615694,0.157360,0.442656,0.074099,0.450704,-0.034277,0.215292,N/A
4,ABMD,319.645,N/A,0.935648,0.975855,0.049355,0.247485,0.203348,0.714286,-0.072291,0.078471,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
492,YUM,104.810,N/A,0.034333,0.394366,0.108285,0.372233,0.009503,0.287726,-0.014723,0.309859,N/A
493,ZBRA,495.720,N/A,1.048366,0.985915,0.773544,0.927565,0.375560,0.901408,0.194379,0.947686,N/A
494,ZBH,162.920,N/A,0.026864,0.380282,0.181566,0.474849,0.100443,0.492958,-0.004161,0.352113,N/A
495,ZION,54.240,N/A,0.218065,0.657948,0.715143,0.901408,0.403055,0.915493,0.116292,0.820926,N/A


In [27]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [28]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,175.420,N/A,0.163132,0.567404,0.107584,0.370221,0.027077,0.325956,0.050469,0.609658,0.46831
1,AOS,61.720,N/A,0.431193,0.851107,0.276189,0.607646,0.108701,0.509054,0.076070,0.698189,0.666499
2,ABT,121.730,N/A,0.423157,0.84507,0.205604,0.511066,0.109601,0.513078,0.086051,0.736419,0.651408
3,ABBV,106.390,N/A,0.187705,0.615694,0.157360,0.442656,0.074099,0.450704,-0.034277,0.215292,0.431087
4,ABMD,319.645,N/A,0.935648,0.975855,0.049355,0.247485,0.203348,0.714286,-0.072291,0.078471,0.504024
...,...,...,...,...,...,...,...,...,...,...,...,...
492,YUM,104.810,N/A,0.034333,0.394366,0.108285,0.372233,0.009503,0.287726,-0.014723,0.309859,0.341046
493,ZBRA,495.720,N/A,1.048366,0.985915,0.773544,0.927565,0.375560,0.901408,0.194379,0.947686,0.940644
494,ZBH,162.920,N/A,0.026864,0.380282,0.181566,0.474849,0.100443,0.492958,-0.004161,0.352113,0.42505
495,ZION,54.240,N/A,0.218065,0.657948,0.715143,0.901408,0.403055,0.915493,0.116292,0.820926,0.823944


In [29]:
newhqmdataframe = hqm_dataframe.sort_values(by=['HQM Score'],ascending=False)

In [33]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

C:\Users\pconn\anaconda3\lib\site-packages\pandas\core\indexing.py:1719: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,169.04,2,-0.015806,0.328713,0.076416,0.253465,0.004136,0.207921,-0.031400,0.128713,0.229703
1,AOS,57.76,6,0.297817,0.762376,0.136540,0.362376,0.056859,0.322772,0.041096,0.560396,0.501980
2,ABT,112.84,3,0.267633,0.720792,0.148784,0.394059,0.051068,0.302970,0.045636,0.590099,0.501980
3,ABBV,110.86,3,0.397986,0.851485,0.170673,0.437624,0.329815,0.831683,0.086158,0.778218,0.724752
4,ABMD,347.90,1,0.901301,0.982178,0.187615,0.475248,0.236231,0.722772,0.146633,0.924752,0.776238
5,ACN,254.77,1,0.240563,0.697030,0.158967,0.413861,0.112933,0.481188,-0.006449,0.263366,0.463861
6,ATVI,94.43,4,0.600186,0.928713,0.180523,0.459406,0.165227,0.611881,0.038149,0.540594,0.635149
7,ADBE,472.44,0,0.344566,0.817822,0.097906,0.289109,-0.032876,0.106931,-0.054855,0.051485,0.316337
8,AAP,163.76,2,0.164863,0.603960,0.115500,0.308911,0.060071,0.330693,0.019169,0.429703,0.418317
9,AMD,92.79,4,0.842900,0.974257,0.337032,0.699010,0.132138,0.536634,0.010674,0.364356,0.643564


In [36]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [37]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [38]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', string_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [39]:
writer.save()

In [17]:
newhqmdataframe = hqm_dataframe.sort_values(by=['HQM Score'],ascending=False)

In [30]:
newhqmdataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
467,VIAC,64.610,N/A,1.436419,0.995976,1.532596,0.993964,0.959071,0.991952,0.451075,0.995976,0.994467
141,DISCA,51.065,N/A,0.893169,0.969819,1.514325,0.991952,1.169086,1.0,0.474793,1.0,0.990443
199,FCX,37.640,N/A,2.189280,1.0,1.651811,0.995976,0.792000,0.987928,0.248934,0.971831,0.988934
142,DISCK,43.270,N/A,0.646975,0.945674,1.295409,0.98994,1.010050,0.997988,0.398601,0.993964,0.981891
444,TWTR,70.565,N/A,0.839990,0.967807,0.795466,0.933602,0.577663,0.973843,0.466708,0.997988,0.96831
...,...,...,...,...,...,...,...,...,...,...,...,...
163,ETR,89.915,N/A,-0.288371,0.032193,-0.075103,0.074447,-0.174434,0.002012,-0.042564,0.187123,0.073944
477,WEC,83.615,N/A,-0.176601,0.088531,-0.095595,0.052314,-0.139519,0.004024,-0.053647,0.138833,0.070926
488,XEL,60.710,N/A,-0.124352,0.142857,-0.120712,0.026157,-0.122421,0.008048,-0.066939,0.092555,0.067404
172,ES,80.100,N/A,-0.158192,0.110664,-0.078817,0.066398,-0.106618,0.020121,-0.100217,0.028169,0.056338


In [32]:
writer = pd.ExcelWriter('momentum_stocks.xlsx', engine='xlsxwriter')
newhqmdataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [33]:
writer.save()